In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix
from keras import backend as K

import os
os.chdir("..")
from src import *


In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


## Load data

In [28]:
_df = pd.read_csv("Data/wig20_d.csv")
# df = pd.read_csv("Data/mwig40_d.csv")
# df = pd.read_csv("Data/swig80_d.csv")

_df.drop(range(1000), inplace=True)
_df.reset_index(inplace=True, drop=True)
_df.head()

,Data,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen
0,1997-02-06,1717.2,1779.8,1717.2,1779.8,4435886.0
1,1997-02-07,1778.2,1808.7,1778.2,1799.0,5448243.0
2,1997-02-10,1797.4,1797.4,1780.3,1783.3,6513315.0
3,1997-02-11,1803.3,1832.2,1803.3,1832.0,5146340.0
4,1997-02-12,1824.6,1824.6,1804.9,1810.2,5748398.0


In [8]:
data = _df.drop(columns=["Data"]).values.transpose()
data.shape

(5, 6429)

## Process and split data

In [9]:
data_pipeline = DataProcess(data, test_ratio=0.2, validation_ratio=0.2, batch_size=32, threshold_fall=-0.007, threshold_rise=0.007)
data_pipeline.run()
_x_train, _y_train, _x_test, _y_test, _x_validation, _y_validation = data_pipeline.get_data()
_y_test_ind, _y_validation_ind = np.argmax(_y_test, axis=1), np.argmax(_y_validation, axis=1)


In [30]:
pd.DataFrame(
    {
        "x": [_x_train.shape, _x_test.shape, _x_validation.shape],
        "y": [_y_train.shape, _y_test.shape, _y_validation.shape]
    },
    index = ["train", "test", "validation"]
)


,x,y
train,"(3825, 5, 32, 2)","(3825, 3)"
test,"(1254, 5, 32, 2)","(1254, 3)"
validation,"(1254, 5, 32, 2)","(1254, 3)"


In [43]:
print("Train dataset classes value counts:")
pd.DataFrame(
    {
        str(key): val for (key, val) in zip(*np.unique(np.argmax(_y_train, axis=1), return_counts=True))
    },
    index = ["count"]
)

Train dataset classes value counts:


,0,1,2
count,1148,1490,1187


## Model

### Building

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.Input(shape=_x_train.shape[1:]))
for i in range(7):
    model.add(tf.keras.layers.Conv2D(2, (1, 3), 1, padding="same"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(1, 3), strides=(1, 3), padding='same'))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(12))
model.add(tf.keras.layers.Dense(7))
model.add(tf.keras.layers.Dense(3, activation="softmax"))
#model.add(tf.keras.layers.Dense(2))


In [14]:
model.build(input_shape=_x_train.shape[1:])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 5, 32, 2)          14        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 5, 11, 2)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 5, 11, 2)          14        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 4, 2)          0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 4, 2)           14        
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 5, 2, 2)          0

In [16]:
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
#               loss=tf.keras.losses.MeanSquaredError())
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=f1_m)
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#               loss=tf.keras.losses.Poisson())


In [18]:
model.fit(_x_train, _y_train, epochs=150)  # with learning_rate=1e-4
# model.fit(_x_train, _y_train, epochs=200)  # with learning_rate=1e-5


Epoch 1/150
120/120 [==============================] - 2s 8ms/step - loss: 279738.2188 - f1_m: 0.3818
Epoch 2/150
120/120 [==============================] - 1s 7ms/step - loss: 14767.5381 - f1_m: 0.3386
Epoch 3/150
120/120 [==============================] - 1s 7ms/step - loss: 12475.2363 - f1_m: 0.3375
Epoch 4/150
120/120 [==============================] - 1s 7ms/step - loss: 11149.2266 - f1_m: 0.3339
Epoch 5/150
120/120 [==============================] - 1s 7ms/step - loss: 9632.4395 - f1_m: 0.3334
Epoch 6/150
120/120 [==============================] - 1s 6ms/step - loss: 9038.2061 - f1_m: 0.3230
Epoch 7/150
120/120 [==============================] - 1s 7ms/step - loss: 7119.8853 - f1_m: 0.3445
Epoch 8/150
120/120 [==============================] - 1s 6ms/step - loss: 6748.4434 - f1_m: 0.3441
Epoch 9/150
120/120 [==============================] - 1s 7ms/step - loss: 5627.0049 - f1_m: 0.3430
Epoch 10/150
120/120 [==============================] - 1s 6ms/step - loss: 4854.7188 - f1_m: 0

In [21]:
# model.save("Models/cdt_2")

### Evaluating

### Test

In [19]:
model.evaluate(_x_test, _y_test)

40/40 [==============================] - 0s 4ms/step - loss: 1.9021 - f1_m: 0.4157


[1.902063250541687, 0.4156889021396637]

In [21]:
pred_test = model.predict(_x_test)
pred_test_ind = np.argmax(pred_test, axis=1)

40/40 [==============================] - 0s 4ms/step


In [22]:
print("Test correct:", np.sum(pred_test_ind == _y_test_ind))
print("Test accuracy:", np.sum(pred_test_ind == _y_test_ind)/pred_test_ind.shape[0])

Test correct: 548
Test accuracy: 0.4370015948963317


In [23]:
confusion_matrix(_y_test_ind, pred_test_ind)

array([[ 29, 210,  37],
       [126, 483,  91],
       [ 38, 204,  36]])

### Valitation

In [20]:
model.evaluate(_x_validation, _y_validation)

40/40 [==============================] - 0s 5ms/step - loss: 1.6282 - f1_m: 0.3883


[1.6281975507736206, 0.38829827308654785]

In [24]:
pred_validation = model.predict(_x_validation)
pred_validation_ind = np.argmax(pred_validation, axis=1)

40/40 [==============================] - 0s 4ms/step


In [25]:
print("Validation correct:", np.sum(pred_validation_ind == _y_validation_ind))
print("Validation accuracy:", np.sum(pred_validation_ind == _y_validation_ind)/pred_validation_ind.shape[0])

Validation correct: 513
Validation accuracy: 0.4090909090909091


In [26]:
confusion_matrix(_y_validation_ind, pred_validation_ind)

array([[ 48, 264,  42],
       [ 87, 404,  81],
       [ 49, 218,  61]])